# Modelos usando el algoritmo de arboles de decisión

# Importación de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import preprocessing
import informe
import sklearn
from sklearn.model_selection import train_test_split
sns.set()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
GSPREADHSEET_DOWNLOAD_URL = (
    "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
)

FIUFIP_2021_1_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=FIUFIP_2021_1_GID))

# Llamado a funciones de preprocesamiento

## Conversión de variables
- Hay que convertir todos los features categóricos a númericos. Para variables categóricas se usarán Dummy Variables y para ordinales OrdinalEncoder.

In [3]:
X = df.drop('tiene_alto_valor_adquisitivo', axis=1)
X = preprocessing.remove_irrelevant_features(X)
X = preprocessing.one_hot_encodding(X)

# Se separa el dataset en entrenamiento y holdout
y = df.tiene_alto_valor_adquisitivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train, scalar = preprocessing.escalar(X_train)

X_test = preprocessing.escalar(X_test, scalar)


X_train.head(20)

       categoria_de_trabajo_empleado_municipal  \
32098                                        0   
25206                                        0   
23491                                        0   
12367                                        0   
7054                                         0   
...                                        ...   
13123                                        1   
19648                                        0   
9845                                         0   
10799                                        1   
2732                                         0   

       categoria_de_trabajo_empleado_provincial  \
32098                                         0   
25206                                         1   
23491                                         0   
12367                                         0   
7054                                          0   
...                                         ...   
13123                                     

/home/sebastian/.local/lib/python3.9/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/sebastian/.local/lib/python3.9/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,categoria_de_trabajo_empleado_municipal,categoria_de_trabajo_empleado_provincial,categoria_de_trabajo_monotibutista,categoria_de_trabajo_relacion_de_dependencia,categoria_de_trabajo_responsable_inscripto,categoria_de_trabajo_sin_trabajo,categoria_de_trabajo_trabajo_voluntariado,estado_marital_matrimonio_civil,estado_marital_matrimonio_militar,estado_marital_pareja_no_presente,...,trabajo_sector_primario,trabajo_seguridad,trabajo_servicio_domestico,trabajo_soporte_tecnico,trabajo_transporte,trabajo_ventas,anios_estudiados,edad,ganancia_perdida_declarada_bolsa_argentina,horas_trabajo_registradas
32098,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
25206,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
23491,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,NaN,NaN,NaN,NaN
12367,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.014816,1.572502,-0.133007,-0.037826
7054,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1.487702,-0.337810,-0.133007,1.589744
369,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,-0.390446,-0.043916,-0.405171,0.287688
10984,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-2.268594,1.425555,-0.133007,-0.037826
30647,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
343,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0.736443,0.837767,-0.133007,-1.665396
23136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN


# Modelos

## Modelo 1

In [4]:
from keras.regularizers import l2


mc = keras.callbacks.ModelCheckpoint('mejor_modelo.h5', monitor='accuracy', mode='max')
custom_early_stopping = keras.callbacks.EarlyStopping(monitor='accuracy', patience=50, mode='max')

model = Sequential()
model.add(Dense(8, input_shape=(40,), activation='tanh', kernel_regularizer=l2(0.001)))
model.add(Dense(1, activation="sigmoid"))

opt = keras.optimizers.RMSprop(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[custom_early_stopping, mc],
    verbose=0
)

Using TensorFlow backend.


AttributeError: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

Gráfico de accuracy de train y test para verificar que no haya overfitting.

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(12, 6), dpi=100)
plt.ylabel("Accuracy")
plt.xlabel("epoc")
plt.plot(history.history["accuracy"], label="training")
plt.plot(history.history["val_accuracy"], label="validation")
plt.legend()

### Métricas


Curva AUC-ROC, Accuracy, Precision, Recall, F1 score:

In [ ]:
from sklearn.metrics import plot_roc_curve

y_proba = model.predict(X_test)
y_pred = model.predict_classes(X_test)
informe.imprimir_metricas_red(y_test, y_pred)
print("Curva ROC-AUC:")
informe.plot_roc(model, X_test, y_test, classifier=False, y_proba=y_proba)

# Conclusión
      
### Informe del modelo 2 usando los datos del test_holdout

In [ ]:
y_pred = model.predict_classes(X_test)

informe.plt_distribucion_de_clases(y_test)
sklearn.metrics.ConfusionMatrixDisplay(sklearn.metrics.confusion_matrix(y_test, y_pred)).plot()
informe.plot_roc(model, X_test, y_test, classifier=False, y_proba=y_proba)
print('Métricas:')
print(sklearn.metrics.classification_report(y_test, y_pred))

## Conclusiones de las métricas observadas de los datos de hold_out

- accuracy:

        El modelo clasifica los datos de forma correcta en aproximadamente un 84%, viendo la distribucion de clases de la muestra se observa que el 0 es la clase mayoritaria con una proporción de aproximadamente 76%. Por lo tanto el modelo es mejor predictor que devolver siempre cero.
    
    
- precisión:

        La fracción de predicciones de 0's que realmente eran 0's fue de aproximadamente 88% y la fracción de predicciones de 1's que realmente eran 1's fue del 71% 


- recall:

        Los 0's reales detectados fueron aproximadamente del 92%, y los 1's reales detectados fueron del 60%. Se ve que el modelo tiene un buen recall y precisión.
        

- f1 score:
   
       Este resultado da una idea de la calidad del recall y la precision asi como el balance entre ellos.
       
       
- matriz de confusion:

        Se puede ver que la cantidad de falsos negativos y falsos positivos fue similar. Sin embargo, la mayoría de las predicciones están en la diagonal.
        
        
- UAC ROC:

        Esta métrica indica que el modelo es bueno distinguiendo clases en un 90%. Este valor será utilizado para decidir sobre la elección de este modelo.


# Final test

In [ ]:
df_test = informe.get_df_test()

#preprocesamiento
df_test = preprocessing.remove_irrelevant_features(df_test)
df_test = preprocessing.one_hot_encodding(df_test)
df_test = scalar.transform(df_test)

y_pred = model.predict_classes(df_test)
informe.save_pred(y_pred, 'red_neuronal')